In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.preprocessing import MinMaxScaler, StandardScaler
sys.path.append('/users/mtaranov/LongRange3D/')
from models_motif import LongRangeDNN
from utils import normalize_features_sampl_by_f, concat_motifs, get_features, get_labels, subsample_data, normalize_features, reconstruct_2d, printMatrix, binarize, zscore, get_2D, plot_prediction, change_scale
#from sklearn.preprocessing import MinMaxScaler, StandardScaler

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 1: GeForce GTX TITAN X (CNMeM is enabled with initial size: 2500 MB, cuDNN 5005)


In [2]:
contacts='pe'
path='/users/mtaranov/datasets_3d/by_chr_dist_matched_'+contacts+'/'
day='d0'
thres='10'

X_train_node1 = path+'motifs/'+day+'_thres_'+thres+'_train_'+contacts+'_out_node1_3_0.0001/mat.npy'
X_train_node2 = path+'motifs/'+day+'_thres_'+thres+'_train_'+contacts+'_out_node2_3_0.0001/mat.npy'
X_train_window = path+'motifs/'+day+'_thres_'+thres+'_train_'+contacts+'_out_btw_nodes_3_0.0001/mat.npy'


X_test_node1 = path+'motifs/'+day+'_thres_'+thres+'_test_'+contacts+'_out_node1_3_0.0001/mat.npy'
X_test_node2 = path+'motifs/'+day+'_thres_'+thres+'_test_'+contacts+'_out_node2_3_0.0001/mat.npy'
X_test_window = path+'motifs/'+day+'_thres_'+thres+'_test_'+contacts+'_out_btw_nodes_3_0.0001/mat.npy'

X_valid_node1 = path+'motifs/'+day+'_thres_'+thres+'_valid_'+contacts+'_out_node1_3_0.0001/mat.npy'
X_valid_node2 = path+'motifs/'+day+'_thres_'+thres+'_valid_'+contacts+'_out_node2_3_0.0001/mat.npy'
X_valid_window = path+'motifs/'+day+'_thres_'+thres+'_valid_'+contacts+'_out_btw_nodes_3_0.0001/mat.npy'

y_train = get_labels(path+day+'_y_train_thres_'+thres+'.npy')
y_test = get_labels(path+day+'_y_test_thres_'+thres+'.npy')
y_valid = get_labels(path+day+'_y_valid_thres_'+thres+'.npy')

X_train=concat_motifs([X_train_node1, X_train_node2, X_train_window])
X_test=concat_motifs([X_test_node1, X_test_node2, X_test_window])
X_valid=concat_motifs([X_valid_node1, X_valid_node2, X_valid_window])

In [3]:
X_train_pairs = X_train.reshape(X_train.shape[0],X_train.shape[2]*X_train.shape[3])
X_valid_pairs = X_valid.reshape(X_valid.shape[0],X_valid.shape[2]*X_valid.shape[3])
X_test_pairs = X_test.reshape(X_test.shape[0],X_test.shape[2]*X_test.shape[3])

In [4]:
X_train_normalized, X_valid_normalized, X_test_normalized = normalize_features_sampl_by_f(X_train_pairs, X_valid_pairs, X_test_pairs)

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [71]:
clf = LogisticRegression(penalty='l1', C=0.009)

In [72]:
clf = clf.fit(X_train_normalized, y_train)

In [73]:
roc_auc_score(y_train, clf.predict_proba(X_train_normalized)[:, 1])

0.91946384269757764

In [74]:
roc_auc_score(y_valid, clf.predict_proba(X_valid_normalized)[:, 1])

0.80964752158967479

In [75]:
roc_auc_score(y_test, clf.predict_proba(X_test_normalized)[:, 1])

0.7757257125997804

In [76]:
clf.coef_.shape

(1, 5760)

In [70]:
w=clf.coef_.reshape(clf.coef_.shape[1])

In [44]:
score_per_feature_node1=w[0:5760:3]
score_per_feature_node2=w[1:5761:3]
score_per_feature_window=w[2:5762:3]

In [45]:
motif_names=np.loadtxt('/users/mtaranov/datasets_3d/dist_matched_pe/motifs/d0_thres_0_test_pe_out_btw_nodes_3_0.0001/motif_names.txt', dtype=str, delimiter='/')[:,6:].reshape(640,)
top_3_hits_names=[]
for i in motif_names:
    for j in range(3):
        top_3_hits_names.append(i)
top_3_hits_names = np.array(top_3_hits_names)

In [46]:
DeepLift_Motifs_node1=top_3_hits_names[score_per_feature_node1.argsort()[::-1]]
DeepLift_Motifs_node2=top_3_hits_names[score_per_feature_node2.argsort()[::-1]]
DeepLift_Motifs_window=top_3_hits_names[score_per_feature_window.argsort()[::-1]]

In [47]:
DeepLift_Motifs_node1[:100]

array(['HXB6_HUMAN', 'ELF3_HUMAN', 'SRY_HUMAN', 'PRGR_HUMAN', 'SOX7_HUMAN',
       'PO6F2_HUMAN', 'ZBT7B_HUMAN', 'ALX4_HUMAN', 'OTX2_HUMAN',
       'HXA7_HUMAN', 'FOXK1_HUMAN', 'FOXK1_HUMAN', 'VDR_HUMAN',
       'PITX1_HUMAN', 'SOX4_HUMAN', 'PO3F1_HUMAN', 'ATOH1_HUMAN',
       'ZBED1_HUMAN', 'ATOH1_HUMAN', 'ESX1_HUMAN', 'ESX1_HUMAN',
       'ESX1_HUMAN', 'ZBED1_HUMAN', 'PO3F1_HUMAN', 'PO3F3_HUMAN',
       'SOX4_HUMAN', 'PO3F3_HUMAN', 'PO3F3_HUMAN', 'ERR2_HUMAN',
       'PO3F1_HUMAN', 'ERR2_HUMAN', 'ERR2_HUMAN', 'STAT2_HUMAN',
       'SOX4_HUMAN', 'ATOH1_HUMAN', 'NKX25_HUMAN', 'STAT2_HUMAN',
       'KLF12_HUMAN', 'NR2E1_HUMAN', 'NR2E1_HUMAN', 'NR2E1_HUMAN',
       'HXD12_HUMAN', 'HXD12_HUMAN', 'HXD12_HUMAN', 'KLF14_HUMAN',
       'KLF14_HUMAN', 'KLF14_HUMAN', 'SOX8_HUMAN', 'SOX8_HUMAN',
       'SOX8_HUMAN', 'KLF12_HUMAN', 'STAT2_HUMAN', 'KLF12_HUMAN',
       'HXA2_HUMAN', 'HXA2_HUMAN', 'HXA2_HUMAN', 'ZEP1_HUMAN',
       'ZEP1_HUMAN', 'MAFK_HUMAN', 'MAFK_HUMAN', 'MAFK_HUMAN',
       'NKX

In [48]:
DeepLift_Motifs_node2[:100]

array(['ETV3_HUMAN', 'ZBT18_HUMAN', 'IRF3_HUMAN', 'OTX2_HUMAN',
       'ITF2_HUMAN', 'ERR2_HUMAN', 'ERR2_HUMAN', 'PO3F3_HUMAN',
       'PO3F3_HUMAN', 'HMGA2_HUMAN', 'ESX1_HUMAN', 'SOX4_HUMAN',
       'ESX1_HUMAN', 'ESX1_HUMAN', 'ATOH1_HUMAN', 'ATOH1_HUMAN',
       'ERR2_HUMAN', 'PO3F1_HUMAN', 'SOX4_HUMAN', 'SOX4_HUMAN',
       'STAT2_HUMAN', 'PO3F1_HUMAN', 'PO3F1_HUMAN', 'ZBED1_HUMAN',
       'ZBED1_HUMAN', 'FOXG1_HUMAN', 'FOXG1_HUMAN', 'FOXG1_HUMAN',
       'PO4F3_HUMAN', 'PO4F3_HUMAN', 'ATOH1_HUMAN', 'STAT2_HUMAN',
       'STAT2_HUMAN', 'KLF12_HUMAN', 'NR2E1_HUMAN', 'NR2E1_HUMAN',
       'NR2E1_HUMAN', 'HXD12_HUMAN', 'HXD12_HUMAN', 'HXD12_HUMAN',
       'KLF14_HUMAN', 'KLF14_HUMAN', 'SOX8_HUMAN', 'SOX8_HUMAN',
       'SOX8_HUMAN', 'KLF12_HUMAN', 'KLF12_HUMAN', 'MLX_HUMAN',
       'HXA2_HUMAN', 'HXA2_HUMAN', 'HXA2_HUMAN', 'ZEP1_HUMAN',
       'ZEP1_HUMAN', 'ZEP1_HUMAN', 'MAFK_HUMAN', 'MAFK_HUMAN',
       'MAFK_HUMAN', 'NKX25_HUMAN', 'NKX25_HUMAN', 'NKX25_HUMAN',
       'PO4F3_HUMAN', 

In [49]:
DeepLift_Motifs_window[:100]

array(['E2F8_HUMAN', 'HXD12_HUMAN', 'MEOX2_HUMAN', 'PAX3_HUMAN',
       'NFAC1_HUMAN', 'ZN423_HUMAN', 'EMX1_HUMAN', 'SOX17_HUMAN',
       'ALX3_HUMAN', 'CPEB1_HUMAN', 'PO3F1_HUMAN', 'HXB1_HUMAN',
       'EMX1_HUMAN', 'MEIS3_HUMAN', 'HSFY1_HUMAN', 'RXRA_HUMAN',
       'RARG_HUMAN', 'ZIC1_HUMAN', 'THAP1_HUMAN', 'SPIC_HUMAN',
       'NKX62_HUMAN', 'PBX3_HUMAN', 'THAP1_HUMAN', 'P63_HUMAN',
       'ARI5B_HUMAN', 'FOXD2_HUMAN', 'PAX8_HUMAN', 'ZFHX3_HUMAN',
       'KLF3_HUMAN', 'THAP1_HUMAN', 'ID4_HUMAN', 'NKX32_HUMAN',
       'CR3L2_HUMAN', 'NANOG_HUMAN', 'LHX8_HUMAN', 'CRX_HUMAN',
       'CEBPD_HUMAN', 'ATF1_HUMAN', 'VSX1_HUMAN', 'EGR2_HUMAN',
       'RARG_HUMAN', 'P73_HUMAN', 'GBX2_HUMAN', 'TAL1_HUMAN', 'EGR3_HUMAN',
       'FOXO3_HUMAN', 'ZSC16_HUMAN', 'BATF_HUMAN', 'DDIT3_HUMAN',
       'CR3L2_HUMAN', 'ESX1_HUMAN', 'VAX2_HUMAN', 'ZIC1_HUMAN',
       'ZN232_HUMAN', 'RARB_HUMAN', 'SOX3_HUMAN', 'TLX1_HUMAN',
       'PDX1_HUMAN', 'CTCF_HUMAN', 'HXD12_HUMAN', 'ESX1_HUMAN',
       'CRX_HUMAN',

In [50]:
N=100
from sets import Set
uniq_node1 = Set(DeepLift_Motifs_node1[:N])
uniq_node2 = Set(DeepLift_Motifs_node2[:N])
uniq_window = Set(DeepLift_Motifs_window[:N])
print len(uniq_node1), len(uniq_node2), len(uniq_window)

45 40 70


In [51]:
uniq_node1 & uniq_node2

Set(['RUNX1_HUMAN', 'OTX2_HUMAN', 'VDR_HUMAN', 'ZBED1_HUMAN', 'NKX25_HUMAN', 'CEBPZ_HUMAN', 'MAFK_HUMAN', 'FOXD3_HUMAN', 'FOXG1_HUMAN', 'STAT2_HUMAN', 'PO4F3_HUMAN', 'PO3F1_HUMAN', 'ATOH1_HUMAN', 'KLF14_HUMAN', 'SOX8_HUMAN', 'HXD12_HUMAN', 'CDC5L_HUMAN', 'ZEP1_HUMAN', 'ERR2_HUMAN', 'ESX1_HUMAN', 'NR2E1_HUMAN', 'SOX4_HUMAN', 'UBIP1_HUMAN', 'KLF13_HUMAN', 'NR4A3_HUMAN', 'IRF7_HUMAN', 'FOXJ3_HUMAN', 'KLF12_HUMAN', 'HXA2_HUMAN', 'SHOX2_HUMAN', 'HMGA2_HUMAN', 'PO3F3_HUMAN'])

In [52]:
uniq_node1 & uniq_node2 & uniq_window

Set(['UBIP1_HUMAN', 'HXD12_HUMAN', 'NR4A3_HUMAN', 'STAT2_HUMAN', 'CDC5L_HUMAN', 'VDR_HUMAN', 'KLF12_HUMAN', 'HXA2_HUMAN', 'ESX1_HUMAN', 'PO3F1_HUMAN', 'NKX25_HUMAN', 'ATOH1_HUMAN', 'RUNX1_HUMAN', 'ZEP1_HUMAN', 'MAFK_HUMAN'])

# Compare with DeepLift

In [59]:
uniq_node1 & Set(['MEOX1_HUMAN', 'TGIF1_HUMAN', 'PHX2B_HUMAN', 'RX_HUMAN',
       'RXRA_HUMAN', 'CEBPZ_HUMAN', 'PRGR_HUMAN', 'MUSC_HUMAN',
       'ZN740_HUMAN', 'HXB8_HUMAN', 'NOBOX_HUMAN', 'HEN1_HUMAN',
       'KLF14_HUMAN', 'ESR2_HUMAN', 'RARA_HUMAN', 'EMX2_HUMAN',
       'MEF2D_HUMAN', 'NR2E3_HUMAN', 'MGAP_HUMAN', 'PO6F1_HUMAN',
       'TBX5_HUMAN', 'HIF1A_HUMAN', 'NDF2_HUMAN', 'COT1_HUMAN',
       'TFDP1_HUMAN', 'MNX1_HUMAN', 'FOXD3_HUMAN', 'ETV2_HUMAN',
       'JUND_HUMAN', 'MEF2D_HUMAN', 'ATF7_HUMAN', 'FOXP2_HUMAN',
       'GLIS3_HUMAN', 'ZN589_HUMAN', 'COT1_HUMAN', 'REL_HUMAN',
       'ELF1_HUMAN', 'TEAD3_HUMAN', 'MLX_HUMAN', 'EGR2_HUMAN', 'P53_HUMAN',
       'DUXA_HUMAN', 'GLIS3_HUMAN', 'VSX2_HUMAN', 'MTF1_HUMAN',
       'TBX3_HUMAN', 'VDR_HUMAN', 'EGR1_HUMAN', 'DMBX1_HUMAN',
       'TEAD3_HUMAN', 'PO4F1_HUMAN', 'ZIC3_HUMAN', 'GABPA_HUMAN',
       'PO3F3_HUMAN', 'FOXO3_HUMAN', 'PBX2_HUMAN', 'KLF14_HUMAN',
       'FLI1_HUMAN', 'FOXC2_HUMAN', 'GFI1_HUMAN', 'FOXM1_HUMAN',
       'HTF4_HUMAN', 'TFE3_HUMAN', 'ESR1_HUMAN', 'TFDP1_HUMAN',
       'PIT1_HUMAN', 'CR3L1_HUMAN', 'GCR_HUMAN', 'CEBPZ_HUMAN',
       'ZN143_HUMAN', 'NFAC1_HUMAN', 'CENPB_HUMAN', 'HEN1_HUMAN',
       'KLF3_HUMAN', 'NR1H2_HUMAN', 'ZBT7B_HUMAN', 'ALX3_HUMAN',
       'DLX5_HUMAN', 'NR5A2_HUMAN', 'RXRB_HUMAN', 'NR4A2_HUMAN',
       'ASCL2_HUMAN', 'GCM1_HUMAN', 'NFAC3_HUMAN', 'COT2_HUMAN',
       'PIT1_HUMAN', 'EGR1_HUMAN', 'MNT_HUMAN', 'SHOX_HUMAN',
       'PO6F2_HUMAN', 'NFE2_HUMAN', 'P73_HUMAN', 'JDP2_HUMAN',
       'LHX6_HUMAN', 'SOX8_HUMAN', 'XBP1_HUMAN', 'BHE23_HUMAN',
       'SNAI2_HUMAN', 'ERR2_HUMAN', 'KLF15_HUMAN'])

Set(['FOXD3_HUMAN', 'SOX8_HUMAN', 'PO6F2_HUMAN', 'VDR_HUMAN', 'ERR2_HUMAN', 'PRGR_HUMAN', 'ZBT7B_HUMAN', 'CEBPZ_HUMAN', 'KLF14_HUMAN', 'PO3F3_HUMAN'])

In [61]:
uniq_node2 & Set(['GATA4_HUMAN', 'ETV2_HUMAN', 'TEAD4_HUMAN', 'HXB6_HUMAN',
       'HXB13_HUMAN', 'FUBP1_HUMAN', 'ZN524_HUMAN', 'HXB6_HUMAN',
       'NR2E3_HUMAN', 'TGIF1_HUMAN', 'ELF1_HUMAN', 'TYY1_HUMAN',
       'PO3F2_HUMAN', 'STA5B_HUMAN', 'HXA13_HUMAN', 'HXD8_HUMAN',
       'ASCL2_HUMAN', 'ELK4_HUMAN', 'ELF3_HUMAN', 'SPDEF_HUMAN',
       'ESR2_HUMAN', 'CEBPB_HUMAN', 'NR2C2_HUMAN', 'ELF5_HUMAN',
       'MEOX1_HUMAN', 'SOX11_HUMAN', 'TAL1_HUMAN', 'NFKB1_HUMAN',
       'RUNX2_HUMAN', 'HBP1_HUMAN', 'NKX21_HUMAN', 'PHX2B_HUMAN',
       'BATF_HUMAN', 'TFDP1_HUMAN', 'HXD8_HUMAN', 'HXD11_HUMAN',
       'TF7L1_HUMAN', 'GMEB2_HUMAN', 'PPARA_HUMAN', 'NR2C2_HUMAN',
       'SP1_HUMAN', 'HMGA1_HUMAN', 'NFAC1_HUMAN', 'ETV3_HUMAN',
       'MITF_HUMAN', 'HMX3_HUMAN', 'NR4A1_HUMAN', 'KLF16_HUMAN',
       'KLF14_HUMAN', 'DLX4_HUMAN', 'XBP1_HUMAN', 'CTCFL_HUMAN',
       'TFDP1_HUMAN', 'ATF3_HUMAN', 'BARX2_HUMAN', 'ERR1_HUMAN',
       'KLF16_HUMAN', 'STAT1_HUMAN', 'HXC12_HUMAN', 'NR1I2_HUMAN',
       'FOXO6_HUMAN', 'TBX3_HUMAN', 'LMX1A_HUMAN', 'COT2_HUMAN',
       'EGR3_HUMAN', 'SHOX2_HUMAN', 'SP3_HUMAN', 'LHX8_HUMAN',
       'CTCF_HUMAN', 'JUND_HUMAN', 'ZN524_HUMAN', 'EOMES_HUMAN',
       'SOX13_HUMAN', 'CREB5_HUMAN', 'ARNT_HUMAN', 'GCR_HUMAN',
       'PITX2_HUMAN', 'COT1_HUMAN', 'HES1_HUMAN', 'ZKSC3_HUMAN',
       'VSX1_HUMAN', 'TBR1_HUMAN', 'CREM_HUMAN', 'NR2F6_HUMAN',
       'KLF13_HUMAN', 'PRD14_HUMAN', 'ZN639_HUMAN', 'P73_HUMAN',
       'HINFP_HUMAN', 'EGR3_HUMAN', 'ELF5_HUMAN', 'P63_HUMAN',
       'LBX2_HUMAN', 'GSX2_HUMAN', 'MLXPL_HUMAN', 'GCM1_HUMAN',
       'MAFG_HUMAN', 'HESX1_HUMAN', 'NANOG_HUMAN', 'MBD2_HUMAN'])

Set(['KLF13_HUMAN', 'SP1_HUMAN', 'KLF14_HUMAN', 'ETV3_HUMAN', 'SHOX2_HUMAN'])